# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [48]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import csv

weatherdata = "weather_data.csv"
weather_df = pd.read_csv(weatherdata)
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ushuaia,-54.8000,-68.3000,44.22,81,40,5.75,AR,1648997654
1,bluff,-46.6000,168.3333,56.95,73,62,11.79,NZ,1648997655
2,rikitea,-23.1203,-134.9692,75.87,64,41,8.79,PF,1648997655
3,yellowknife,62.4560,-114.3525,5.41,80,20,11.50,CA,1648997656
4,mataura,-46.1927,168.8643,45.30,98,53,1.83,NZ,1648997415
...,...,...,...,...,...,...,...,...,...
541,vanimo,-2.6741,141.3028,78.33,85,99,0.65,PG,1648998487
542,sedalia,38.7045,-93.2283,57.85,43,0,13.00,US,1648998488
543,mhlambanyatsi,-26.4500,31.0167,60.06,97,100,0.27,SZ,1648998488
544,romny,50.7510,33.4747,35.94,85,100,8.30,UA,1648998488


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
import gmaps

locations = weather_df[["Lat","Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity,
                                 dissipating=False, 
                                 max_intensity=weather_df["Humidity"].max(),
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [83]:
#A max temperature lower than 80 (26.67 Celsius) degrees but higher than 65 (21.11 Celsius).
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_vac_df = weather_df.loc[weather_df["Max Temp"] < 80 ]
ideal_vac_df = ideal_vac_df.loc[ideal_vac_df["Max Temp"] > 65 ]
ideal_vac_df = ideal_vac_df[ideal_vac_df["Cloudiness"] == 0 ]
ideal_vac_df = ideal_vac_df.loc[ideal_vac_df["Wind Speed"] < 10 ]


ideal_df = ideal_vac_df.reset_index(drop=True)
ideal_df 



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saldanha,-33.0117,17.9442,77.05,65,0,7.81,ZA,1648997659
1,callaway,30.1530,-85.5699,70.92,55,0,8.99,US,1648997659
2,san patricio,28.0170,-97.5169,75.92,65,0,7.40,US,1648997665
3,villa carlos paz,-31.4241,-64.4978,76.06,32,0,4.00,AR,1648997587
4,russell,32.3502,-85.1999,67.05,55,0,4.61,US,1648997716
5,wilmington,34.2257,-77.9447,68.00,55,0,3.00,US,1648997796
6,mount isa,-20.7333,139.5000,71.37,33,0,0.00,AU,1648998027
7,komsomolskiy,40.4272,71.7189,68.25,31,0,5.84,UZ,1648998034
8,mangrol,21.1167,70.1167,79.00,76,0,8.70,IN,1648998479


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [88]:
#Create list of hotels

hotels = []
hotels_lat = []
hotels_lng = []
hotels_city = []
hotels_country = []



base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


#set variables
for i in range(len(ideal_df['City'])):
    target_radius = 5000
    target_type = "hotel"
    target_coordinates = f'{ideal_df["Lat"][i]},{ideal_df["Lng"][i]}'
    params= {
        "location":target_coordinates,
        "keyword" : "Hotel",
        "radius":target_radius,
        "type":target_type,
        "key":g_key
    }
    response = requests.get(base_url, params=params).json()
    hotels.append(response["results"][0]["name"])
    hotels_lat.append(response["results"][0]["geometry"]["location"]["lat"])
    hotels_lng.append(response["results"][0]["geometry"]['location']["lng"])
    hotels_city.append(ideal_df["City"][i])
    hotels_country.append(ideal_df["Country"][i])

In [89]:
hotel_df = pd.DataFrame({"Hotel Name" : hotels,
                           "Lat" : hotels_lat,
                           "Lng" : hotels_lng,
                           "City" : hotels_city,
                           "Country" : hotels_country})
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Saldanha Bay Hotel,-33.009115,17.947599,saldanha,ZA
1,Days Inn by Wyndham Panama City/Callaway,30.155940,-85.591564,callaway,US
2,Americas Best Value Inn Sinton,28.028582,-97.529505,san patricio,US
3,Howard Johnson by Wyndham Villa Carlos Paz,-31.408445,-64.485763,villa carlos paz,AR
4,Macs,32.360373,-85.177116,russell,US
5,"Hotel Ballast Wilmington, Tapestry Collection ...",34.238827,-77.950806,wilmington,US
6,Isa Hotel,-20.726329,139.491381,mount isa,AU
7,Diyor hotel,40.441685,71.716445,komsomolskiy,UZ
8,National Hotel,21.121589,70.120486,mangrol,IN


In [90]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [91]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))